# Reaction essentiality

In [1]:
import pandas as pd
from cobra.io.json import load_json_model
import numpy as np
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import single_reaction_deletion
import math
from tqdm import tqdm

from cobra.io import read_sbml_model
from mewpy.simulation import get_simulator
from mewpy.simulation import set_default_solver
set_default_solver('gurobi')

In [7]:
def rxn_essential_df(model):
    df_res_rxn = single_reaction_deletion(model, processes=10)
    df_essential=df_res_rxn[df_res_rxn["status"]!="optimal"] # remove non essential
    df_essential["rxn_ids"]=df_essential.ids.apply(lambda x: list(x)[0] if isinstance(x, (set, frozenset)) else x) # remove {} around ids

    return df_essential

In [8]:
def rxn_essential(model):
    
    df_essential = rxn_essential(model)

    def change_bounds(rxn_id):
        if rxn_id in model.reactions:
            rxn= model.reactions.get_by_id(rxn_id)
            lb,ub= rxn.lower_bound, rxn.upper_bound
            M=1000

            if ub <= 0 and lb < 0:
                rxn_bounds=(-M, 0.0)
            elif lb >= 0 and ub > 0:
                rxn_bounds=(0.0, +M)
            else:
                rxn_bounds=(-M, +M)
        
            
            rxn.bounds=rxn_bounds
        else:
            print("error reaction doesn't exist")
    
    df_essential["rxn_ids"].apply(change_bounds)


In [2]:
model = load_json_model('./data/Recon3D.json')
model.solver = 'gurobi' #you can use cplex or glpk as you whish, but gurobi and cplex are usually faster


Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04


In [3]:
model_HG= model.copy()

Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmp8bzhq50d.lp
Reading time = 0.02 seconds
: 5835 rows, 21200 columns, 80850 nonzeros


In [4]:
df_fva_HG  = pd.read_csv('./data/fva_exoflux_HG.csv', index_col=0)

#inject bounds into the model
for index, row in df_fva_HG.iterrows():
    model_HG.reactions.get_by_id(index).bounds = (row['minimum'], row['maximum'])


In [10]:
df_essential= rxn_essential_df(model_HG)

Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmp0lao9ksv.lp
Reading time = 0.03 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmpkt6epahm.lp
Reading time = 0.03 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmp87z5lmkj.lp
Reading time = 0.02 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use 

/var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/ipykernel_50441/3727464996.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_essential["rxn_ids"]=df_essential.ids.apply(lambda x: list(x)[0] if isinstance(x, (set, frozenset)) else x) # remove {} around ids


In [16]:
df_essential

,ids,growth,status,rxn_ids
276,{34HPPte},NaN,infeasible,34HPPte
305,{EX_dmgly_e},NaN,infeasible,EX_dmgly_e
497,{SUCCt4_2},NaN,infeasible,SUCCt4_2
829,{EX_g3pc_e},NaN,infeasible,EX_g3pc_e
1128,{EX_cyst__L_e},NaN,infeasible,EX_cyst__L_e
2068,{EX_mal__L_e},NaN,infeasible,EX_mal__L_e
2270,{EX_fum_e},NaN,infeasible,EX_fum_e
2535,{XANt},NaN,infeasible,XANt
2928,{LPASE},NaN,infeasible,LPASE
3029,{EX_glyc3p_e},NaN,infeasible,EX_glyc3p_e
